In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('seaborn')
mpl.rc('font',family='Malgun Gothic') # 한글폰트 설정
mpl.rcParams['axes.unicode_minus'] = False # 마이너스 단위 설정

In [2]:
import pandas as pd
import mariadb
import sys

# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="root",
        password="root",
        host="34.64.93.150",
        port=3306,
        database="smart_factory"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

cur = conn.cursor()
cur.execute(f"SELECT 원자재코드, 생산작업요청일자, LOT번호, 제품코드, 제품BOM차수, 투입지시중량, 투입지시비율 FROM `production_recipe`")
x = cur.fetchall()
df = pd.DataFrame(x, columns=['원자재코드', '생산작업요청일자', 'LOT번호', '제품코드', '제품BOM차수', '투입지시중량', '투입지시비율'])
df

,원자재코드,생산작업요청일자,LOT번호,제품코드,제품BOM차수,투입지시중량,투입지시비율
0,SR11002,2021-02-22,580F21022209,PEMA033101400,25,450,3
1,SR35218,2021-02-22,580F21022209,PEMA033101400,25,13095,87
2,SR35904,2021-02-22,580F21022209,PEMA033101400,25,1455,10
3,SR11002,2021-02-22,580F21022215,PEMA023098200,65,1800,9
4,SR35218,2021-02-22,580F21022215,PEMA023098200,65,18200,91
...,...,...,...,...,...,...,...
12299,SR35217,2021-04-22,SR2021042141,PEMA013123000,52,1200,12
12300,SR35218,2021-04-22,SR2021042141,PEMA013123000,52,1860,19
12301,SR35221,2021-04-22,SR2021042141,PEMA013123000,52,630,6
12302,SR35758,2021-04-22,SR2021042141,PEMA013123000,52,200,2


In [3]:
df1 = df.copy()
for i in df1.columns.tolist():
    df1[i] = df1[i].astype('str')
df1['임시코드'] = df1['생산작업요청일자'] + df1['LOT번호'] + df1['제품코드'] + df1['제품BOM차수']
df1[['임시코드','원자재코드','투입지시중량']]

,임시코드,원자재코드,투입지시중량
0,2021-02-22580F21022209PEMA03310140025,SR11002,450
1,2021-02-22580F21022209PEMA03310140025,SR35218,13095
2,2021-02-22580F21022209PEMA03310140025,SR35904,1455
3,2021-02-22580F21022215PEMA02309820065,SR11002,1800
4,2021-02-22580F21022215PEMA02309820065,SR35218,18200
...,...,...,...
12299,2021-04-22SR2021042141PEMA01312300052,SR35217,1200
12300,2021-04-22SR2021042141PEMA01312300052,SR35218,1860
12301,2021-04-22SR2021042141PEMA01312300052,SR35221,630
12302,2021-04-22SR2021042141PEMA01312300052,SR35758,200


In [4]:
raw_li = df.원자재코드.unique().tolist()
raw_li1 = ['SR11002',
       'SR13801', 'SR15500', 'SR16002', 'SR16003', 'SR16600', 'SR18520',
       'SR34307', 'SR34311', 'SR34364', 'SR35210', 'SR35211', 'SR35217',
       'SR35218', 'SR35221', 'SR35224', 'SR35225', 'SR35228', 'SR35500',
       'SR35556', 'SR35755', 'SR35758', 'SR35760', 'SR35761', 'SR35765',
       'SR35903', 'SR35904', 'SR35905', 'SR38117', 'SR47093', 'SR68222',
       'PEMA023017400','SSA043031400','TSN4012039104','TSN4012057700']
raw_li2 = ['PEMA013010900', 'PEMA013022400', 'PEMA013052200', 'PEMA013072100',
       'PEMA013081906', 'PEMA013112900', 'PEMA013130400',
       'PEMA023031800', 'PEMA023038000', 'PEMA023126100', 'PEMA023132100',
       'PEMA033035307', 'PEMA043023200', 'PEMA043045700', 'PEMA043062007',
       'PEMA043091500', 'PEMA053006900', 'PEMA313001100', 'TSN4012043602']

In [5]:
df_raw = pd.DataFrame(df1[['임시코드','원자재코드','투입지시중량']].임시코드.unique().tolist(), columns=['임시코드'])

for i in raw_li:
    df_raw[i] = 0
for i in raw_li:
    for j in df_raw.임시코드.tolist():
        try:
            df_raw.loc[(df_raw.임시코드 == j),i] = df1.loc[(df1.임시코드 == j)&(df1.원자재코드 == i),'투입지시중량'].values[0]
        except:
            df_raw.loc[df_raw.임시코드 == j,i] = 0

for i in df_raw.columns.tolist()[1:]:
    df_raw[i] = df_raw[i].astype('int')

df_raw['total'] = df_raw.drop('임시코드',axis=1).sum(axis=1)

for i in df_raw.columns.tolist()[1:]:
    df_raw[i] = df_raw[i] / df_raw['total']
df_raw = df_raw.merge(df1[['임시코드','제품코드']].drop_duplicates(), on='임시코드', how='left')

df_raw

,임시코드,SR11002,SR35218,SR35904,SR15500,SR13801,SR16600,SR35224,SR35903,SR68222,...,SSA043031400,PEMA013112900,PEMA043062007,PEMA053006900,PEMA013130400,PEMA043091500,PEMA023031800,PEMA023126100,total,제품코드
0,2021-02-22580F21022209PEMA03310140025,0.030000,0.873,0.097,0.0,0.0000,0.000000,0.000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA033101400
1,2021-02-22580F21022215PEMA02309820065,0.090000,0.910,0.000,0.0,0.0000,0.000000,0.000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA023098200
2,2021-02-22AE21022240SSA073046500287,0.000000,0.000,0.000,1.0,0.0000,0.000000,0.000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,SSA073046500
3,2021-02-22CSA521022218PEMA04309980044,0.664000,0.000,0.149,0.0,0.0005,0.027500,0.118,0.04,0.0010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA043099800
4,2021-02-22CSA521022232PEMA01312000015,0.683500,0.000,0.085,0.0,0.0005,0.060000,0.140,0.03,0.0010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA013120000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672,2021-04-21SR2021042139PEMA0431038003,0.661600,0.000,0.000,0.0,0.0010,0.005500,0.120,0.04,0.0012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA043103800
1673,2021-04-21SR2021042143PEMA02312240021,0.618000,0.000,0.000,0.0,0.0005,0.010000,0.096,0.00,0.0010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA023122400
1674,2021-04-22HR1021042132PEMA353048300199,0.657857,0.000,0.000,0.0,0.0010,0.008143,0.140,0.05,0.0010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA353048300
1675,2021-04-22SN4021042142TSN40120111251,0.659300,0.000,0.000,0.0,0.0000,0.000000,0.000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,TSN4012011125


In [6]:
df_raw['total_raw2'] = df_raw[raw_li2].sum(axis=1)
df_raw1 = df_raw[df_raw.total_raw2 == 0].drop(['임시코드','total_raw2'],axis=1).groupby('제품코드').mean()
df_raw1 = df_raw1.drop(raw_li2,axis=1).reset_index()
df_raw1

,제품코드,SR11002,SR35218,SR35904,SR15500,SR13801,SR16600,SR35224,SR35903,SR68222,...,SR34311,SR35211,TSN4012039104,SR35221,SR16003,PEMA023017400,TSN4012057700,SR47093,SSA043031400,total
0,PCA021027605,0.000000,0.854971,0.145029,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,PCA031024602,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,PEMA003039101,0.718050,0.000000,0.015000,0.0,0.000000,0.019000,0.206250,0.00,0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,PEMA013000210,0.669000,0.000000,0.127500,0.0,0.001000,0.016500,0.115000,0.06,0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,PEMA013002500,0.691641,0.000000,0.019997,0.0,0.000533,0.016531,0.057992,0.00,0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,TSN4012039105,0.659300,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
597,TSN4012043601,0.659268,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
598,TSN4012043602,0.659300,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
599,TSN4012056000,0.659286,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
df_raw2 = df_raw[df_raw.total_raw2 != 0]
df_raw2.head()

,임시코드,SR11002,SR35218,SR35904,SR15500,SR13801,SR16600,SR35224,SR35903,SR68222,...,PEMA013112900,PEMA043062007,PEMA053006900,PEMA013130400,PEMA043091500,PEMA023031800,PEMA023126100,total,제품코드,total_raw2
158,2021-02-26CSA521022529PEMA013072700200,0.524900,0.0,0.120000,0.0,0.0009,0.037000,0.07940,0.0370,0.0008,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA013072700,0.200000
165,2021-02-26HR1021022608PEMA013010100196,0.462400,0.0,0.195000,0.0,0.0010,0.124500,0.04970,0.0365,0.0009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA013010100,0.100000
176,2021-02-26SR2021022601PEMA03304930124,0.055389,0.0,0.131774,0.0,0.0008,0.003799,0.09938,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA033049301,0.657868
225,2021-03-02HR1021030220PEMA01312160028,0.512600,0.0,0.125000,0.0,0.0009,0.008000,0.00700,0.0560,0.0005,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA013121600,0.200000
244,2021-03-03CSA521030304PEMA063069401105,0.271500,0.0,0.260000,0.0,0.0008,0.115000,0.03180,0.0037,0.0002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA063069401,0.317000


In [8]:
for j in df_raw2.index:
    for i in raw_li2:
        if df_raw2.loc[j,i] != 0:
            temp_rate = df_raw2.loc[j,i]
            df_raw2.loc[j,raw_li1] = df_raw2.loc[j,raw_li1] + (df_raw1.loc[df_raw1.제품코드 == raw_li2[0],raw_li1].values[0] * temp_rate)
df_raw2.head()

C:\Users\chonh\anaconda3\envs\cakd5\lib\site-packages\pandas\core\indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


,임시코드,SR11002,SR35218,SR35904,SR15500,SR13801,SR16600,SR35224,SR35903,SR68222,...,PEMA013112900,PEMA043062007,PEMA053006900,PEMA013130400,PEMA043091500,PEMA023031800,PEMA023126100,total,제품코드,total_raw2
158,2021-02-26CSA521022529PEMA013072700200,0.670455,0.0,0.120000,0.0,0.001100,0.040250,0.101400,0.048000,0.001000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA013072700,0.200000
165,2021-02-26HR1021022608PEMA013010100196,0.535177,0.0,0.195000,0.0,0.001100,0.126125,0.060700,0.042000,0.001000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA013010100,0.100000
176,2021-02-26SR2021022601PEMA03304930124,0.534169,0.0,0.131774,0.0,0.001458,0.014490,0.171746,0.036183,0.000658,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA033049301,0.657868
225,2021-03-02HR1021030220PEMA01312160028,0.658155,0.0,0.125000,0.0,0.001100,0.011250,0.029000,0.067000,0.000700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA013121600,0.200000
244,2021-03-03CSA521030304PEMA063069401105,0.502205,0.0,0.260000,0.0,0.001117,0.120151,0.066670,0.021135,0.000517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PEMA063069401,0.317000


In [9]:
df_raw2 = df_raw2.drop(raw_li2,axis=1)
df_raw2 = df_raw2.drop(['임시코드','total_raw2'],axis=1).groupby('제품코드').mean().reset_index()
df_raw2.head()

,제품코드,SR11002,SR35218,SR35904,SR15500,SR13801,SR16600,SR35224,SR35903,SR68222,...,SR34311,SR35211,TSN4012039104,SR35221,SR16003,PEMA023017400,TSN4012057700,SR47093,SSA043031400,total
0,PEMA013010100,0.535177,0.0,0.195000,0.0,0.001100,0.126125,0.060700,0.04200,0.001000,...,0.0,0.0,0.0,0.002048,0.000100,0.0,0.0,0.0,0.0,1.0
1,PEMA013021700,0.765755,0.0,0.100000,0.0,0.035100,0.011750,0.032000,0.03600,0.001200,...,0.0,0.0,0.0,0.004095,0.000600,0.0,0.0,0.0,0.0,1.0
2,PEMA013030801,0.713443,0.0,0.000000,0.0,0.000530,0.033287,0.144300,0.06865,0.001030,...,0.0,0.0,0.0,0.008804,0.000930,0.0,0.0,0.0,0.0,1.0
3,PEMA013035322,0.530111,0.0,0.144714,0.0,0.000571,0.068321,0.194571,0.04500,0.001143,...,0.0,0.0,0.0,0.002925,0.000143,0.0,0.0,0.0,0.0,1.0
4,PEMA013039505,0.608507,0.0,0.083571,0.0,0.001000,0.101786,0.098714,0.06000,0.001000,...,0.0,0.0,0.0,0.005850,0.001714,0.0,0.0,0.0,0.0,1.0


In [37]:
df_raw_final = pd.concat([df_raw1,df_raw2]).drop('total',axis=1)
df_raw_final = df_raw_final.groupby('제품코드').mean().reset_index()
df_raw_final

,제품코드,SR11002,SR35218,SR35904,SR15500,SR13801,SR16600,SR35224,SR35903,SR68222,...,SR35765,SR34311,SR35211,TSN4012039104,SR35221,SR16003,PEMA023017400,TSN4012057700,SR47093,SSA043031400
0,PCA021027605,0.000000,0.854971,0.145029,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,PCA031024602,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,PEMA003039101,0.718050,0.000000,0.015000,0.0,0.000000,0.019000,0.206250,0.00,0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,PEMA013000210,0.669000,0.000000,0.127500,0.0,0.001000,0.016500,0.115000,0.06,0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,PEMA013002500,0.691641,0.000000,0.019997,0.0,0.000533,0.016531,0.057992,0.00,0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,TSN4012039105,0.659300,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
604,TSN4012043601,0.659268,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
605,TSN4012043602,0.659300,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
606,TSN4012056000,0.659286,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
import pandas as pd
import mariadb
import sys

# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="root",
        password="root",
        host="34.64.93.150",
        port=3306,
        database="smart_factory"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

cur = conn.cursor()
cur.execute(f"SELECT 제품명, 제품코드 FROM `production_re`")
x = cur.fetchall()
df_prodname = pd.DataFrame(x, columns=['제품명','제품코드'])
df_prodname

,제품명,제품코드
0,ARE-580FX,PCA021027605
1,SRE-110,PCA031024602
2,CSA5000,PEMA003039101
3,PEMA-HR1000,PEMA013000210
4,PEMA-HR1500,PEMA013002500
...,...,...
1211,PEMA-SN400,TSN4012039105
1212,PEMA-SN400,TSN4012043601
1213,PEMA-SN400,TSN4012043602
1214,PEMA-SN400,TSN4012056000


In [39]:
df_raw_final = df_raw_final.merge(df_prodname, on='제품코드', how='left').drop('제품코드',axis=1)
df_raw_final = df_raw_final[['제품명'] + df_raw_final.columns[:-1].tolist()].groupby('제품명').mean().reset_index()
# df_raw_final[df_raw_final.columns.tolist()[1:]] = df_raw_final[df_raw_final.columns.tolist()[1:]].apply(lambda x: x.round(4))
df_raw_final

,제품명,SR11002,SR35218,SR35904,SR15500,SR13801,SR16600,SR35224,SR35903,SR68222,...,SR35765,SR34311,SR35211,TSN4012039104,SR35221,SR16003,PEMA023017400,TSN4012057700,SR47093,SSA043031400
0,AE,0.277000,0.000000,0.000000,0.688000,0.000000,0.035000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,ARE-580FX,0.000000,0.854971,0.145029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
2,CSA4000,0.708263,0.028729,0.080518,0.000000,0.006635,0.037419,0.063348,0.040074,0.000915,...,0.000000,0.000000,0.000000,0.000000,0.000307,0.000450,0.000000,0.000000,0.00000,0.000000
3,CSA5000,0.667611,0.001930,0.100486,0.000000,0.002611,0.038594,0.099582,0.035795,0.001090,...,0.000000,0.000000,0.003161,0.000000,0.004632,0.000557,0.001458,0.000000,0.00000,0.000525
4,CSA5000F,0.695150,0.000000,0.086500,0.000000,0.000750,0.019000,0.120000,0.052500,0.000850,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000250,0.000000,0.000000,0.00000,0.000000
5,PEMA-500FR,0.059279,0.545374,0.000000,0.000000,0.001521,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.058173,0.000000,0.000000,0.000000,0.00000,0.000000
6,PEMA-580FX,0.077374,0.864250,0.016167,0.000000,0.001420,0.000000,0.037666,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
7,PEMA-CR1000,0.653143,0.055186,0.000000,0.000000,0.001000,0.209421,0.030000,0.025000,0.001107,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000143,0.000000,0.000000,0.00000,0.000000
8,PEMA-CSA5000,0.660167,0.000000,0.110000,0.000000,0.000500,0.015500,0.136000,0.060000,0.001000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000167,0.000000,0.000000,0.00000,0.000000
9,PEMA-HR1000,0.643702,0.011120,0.084817,0.000107,0.001618,0.026540,0.111210,0.032419,0.001023,...,0.000000,0.000759,0.009002,0.000000,0.003176,0.000398,0.000000,0.000000,0.00000,0.000029


In [40]:
df_raw_final.drop('제품명',axis=1).sum(axis=1)

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
31    1.0
dtype: float64

In [41]:
df_raw_final.to_csv('./dataset/recipe.csv', index=False)

In [42]:
import pandas as pd
pd.read_csv('./dataset/recipe.csv')

,제품명,SR11002,SR35218,SR35904,SR15500,SR13801,SR16600,SR35224,SR35903,SR68222,...,SR35765,SR34311,SR35211,TSN4012039104,SR35221,SR16003,PEMA023017400,TSN4012057700,SR47093,SSA043031400
0,AE,0.277000,0.000000,0.000000,0.688000,0.000000,0.035000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,ARE-580FX,0.000000,0.854971,0.145029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
2,CSA4000,0.708263,0.028729,0.080518,0.000000,0.006635,0.037419,0.063348,0.040074,0.000915,...,0.000000,0.000000,0.000000,0.000000,0.000307,0.000450,0.000000,0.000000,0.00000,0.000000
3,CSA5000,0.667611,0.001930,0.100486,0.000000,0.002611,0.038594,0.099582,0.035795,0.001090,...,0.000000,0.000000,0.003161,0.000000,0.004632,0.000557,0.001458,0.000000,0.00000,0.000525
4,CSA5000F,0.695150,0.000000,0.086500,0.000000,0.000750,0.019000,0.120000,0.052500,0.000850,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000250,0.000000,0.000000,0.00000,0.000000
5,PEMA-500FR,0.059279,0.545374,0.000000,0.000000,0.001521,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.058173,0.000000,0.000000,0.000000,0.00000,0.000000
6,PEMA-580FX,0.077374,0.864250,0.016167,0.000000,0.001420,0.000000,0.037666,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
7,PEMA-CR1000,0.653143,0.055186,0.000000,0.000000,0.001000,0.209421,0.030000,0.025000,0.001107,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000143,0.000000,0.000000,0.00000,0.000000
8,PEMA-CSA5000,0.660167,0.000000,0.110000,0.000000,0.000500,0.015500,0.136000,0.060000,0.001000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000167,0.000000,0.000000,0.00000,0.000000
9,PEMA-HR1000,0.643702,0.011120,0.084817,0.000107,0.001618,0.026540,0.111210,0.032419,0.001023,...,0.000000,0.000759,0.009002,0.000000,0.003176,0.000398,0.000000,0.000000,0.00000,0.000029
